# CNN (Convolutional Neural Network) : 합성곱 신경망

#### FC Layer (Fully Connected Layer)
- 이전 layer의 모든 node가 다음 layer의 모든 node에 연결되어 학습되는 layer 구조
- 다른 말로 Dense Layer라고도 한다.
- 지금까지 우리가 작업한 신경망은 모두 FC Layer를 이용하고 있음 
- 지금까지 우리가 배운 내용 (모든 레이어가 연결되어 그 다음 입력값으로 입력되는 구조)  

#### FC Layer의 특징: MNIST의 예제처럼 입력 데이터가 1차원으로 한정된다.
( 한사람의 데이터가 1차원으로 표현 -> 여러사람이라 2차원 형태였음)   
(image도 흑백이라 2차원 자료)  
(즉, 각각의 이미지가 1차원으로 표현되어야 한다.)
- 그래서 2차원 이미지를 우리가 1차원으로 변환시켜서 사용한 것
- 우리가 사용한 MNIST 예제는 상당히 간단한 이미지 학습, 예측 예제  

#### FC Layer의 문제  
- 이미지 학습의 가장 큰 문제는 이미지가 살짝 휘어있거나 크기가 제각각이거나 변형이 조금만 생겨도 학습이 힘들어진다. 
- 이런 경우에는 training data가 굉장히 많이 필요하고 추가적으로 학습할 때 많은 시간을 요구하게 된다. 
- 고민하면서 방법을 연구하기 시작
- 사람이 학습하는 방식을 모델링함 -> 특징을 가지고 학습
- 찾아낸 방법: 이미지의 픽셀값을 그대로 입력하는게 아니라 이미지를 대표하는 특징을 도출해서 신경망에 여러개 넣어서 학습하는 방식  

#### CNN
- 1장의 컬러사진은 width, height, color(depth) 3차원으로 표현
- 여러장의 사진이 사용되기 때문에 입력 데이터는 4차원으로 표현 
- 실제 이미지 1장은 3차원이고 이걸 flatten 시켜서 1차원으로 표현해야 한다.
- 크기를 조절해야 하기 때문에 공간에 대한 데이터를 유실할 우려가 있다. 
- 이런 데이터 유실때문에 학습과 예측에 문제가 발생하게 된다.  

#### 공간 데이터의 유실을 없애고 이미지의 특성을 추출해서 학습이 용이하게 만드는 방식 => CNN 
- 이렇게 만든 데이터로 FC Layer에 넣는 것
- CNN은 우리가 배운거 하기 전에 끼워넣는 단계 




# 코드로 알아보자! 
### 사용되는 함수부터 알아보자. 
### sample CNN






In [2]:
import tensorflow as tf
import numpy as np

# 입력 데이터의 형식: 3*3*1 (width*height*color) 이미지 1개를 이용
# 입력 데이터 -> 이미지 개수, width, height, color 수 -> (1,3,3,1)
# 총 9개의 데이터 사용(1~9)
image = np.array([[[[1],[2],[3]],
                   [[4],[5],[6]],
                   [[7],[8],[9]]]], dtype=np.float32)
print("image의 shape : {}".format(image.shape )) # (1, 3, 3, 1) 마지막 값은 []괄호 안의 값 몇 개냐

# Activation map을 위한 filter를 정의 (width, height, color, filter 개수)
# filter (2,2,1,3) : 2행 2열, depth(color 개수)는 1, filter개수는 3  
weight = np.array([[[[1,10,-1]],[[2,10,-1]]],
                   [[[1,10,-1]],[[1,10,-1]]]])
print("weight(filter)의 shape : {}".format(weight.shape))

# stride = 1 (가로, 세로를 1씩 움직여라)
conv2d = tf.nn.conv2d(image, weight, strides=[1,1,1,1], padding="VALID")   
    # padding="VALID": 패딩하지 마라(사이즈 줄이겠다)  / padding="SAME" 하면 차원 같게 (패딩처리 해라)
    # [1,가로, 세로,1]  
    # stride크기는 가로 세로 같게 잡는것이 좋음. # 양 옆의 1,1은 4차원 만들어주는 더미변수일 뿐
print("conv2d의 shape : {}".format(conv2d))        # (1, 2, 2, 3): (2,2,3)이 1장 있다 
sess = tf.Session()
conv2d = sess.run(conv2d)

# pooling layer 
pool = tf.nn.max_pool(conv2d, ksize=[1,2,2,1],   # 2,2 크기의 빈칸 
                      strides=[1,1,1,1], padding="SAME") # ksize: kernel size  # 앞뒤값은 더미. 가운데가 중요!
        # SAME의 의미는 padding을 상하좌우에 붙인다는게 아니라 출력 크기가 처음과 같게 해주도록 붙여준다는 것
print("pool의 shape : {}".format(pool.shape))   # (1, 2, 2, 3)


image의 shape : (1, 3, 3, 1)
weight(filter)의 shape : (2, 2, 1, 3)
conv2d의 shape : Tensor("Conv2D_1:0", shape=(1, 2, 2, 3), dtype=float32)
pool의 shape : (1, 2, 2, 3)


In [2]:
# convolution 결과 이미지가 원본 이미지에 비해 어떻게 다른지 눈으로 확인해보자!
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

# data loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

img = mnist.train.images[0].reshape(28,28)
plt.imshow(img, cmap="Greys")

# 해당 이미지를 convolution 처리를 해 보자!
# 입력 데이터의 형식: 3*3*1 (width*height*color) 
img = img.reshape(-1,28,28,1)  # 맨 앞은 이미지 개수이므로 1이라고 써도 되지만 -1쓰면 뒤에거 채우고 나머지 다 이미지 개수로 채움
# Activation map을 위한 filter를 정의 (width, height, color, filter 개수)
W = tf.Variable(tf.random_normal([3,3,1,5]),name="filter1")
conv2d = tf.nn.conv2d(img, W, strides=[1,2,2,1], padding="SAME") 
    # strides가 1일 경우, padding="SAME" 하면 원래 크기와 같게 나오지만, ->28,28
    # strides가 2일 경우, padding="SAME" 하면 원래 크기의 절반 크기로 출력됨 -> 14,14
print("conv2d의 shape : {}".format(conv2d.shape))   # (1,14,14,5)

sess = tf.Session()
sess.run(tf.global_variables_initializer())  # 앞에서 랜덤으로 잡는 거 있었으므로 초기화 시켜줘야 한다.
conv2d = sess.run(conv2d)

# (1,14,14,5) -> (14,14)크기의 5장이 1개 있다
# (5,14,14,1) -> (14,14,1)크기의 5장이 있다 
# -> 같은 말로 해석 가능
# 뒷부분 3장이 이미지 하나를 나타내는 말이므로 

# 이미지를 표현하기 위해서 축을 변환해보자!
# (1,14,14,5) -> (5,14,14,1) 
conv2d_img = np.swapaxes(conv2d,0,3)
print("conv2d_img의 shape : {}".format(conv2d_img.shape))   

plt.imshow(conv2d_img[0].reshape(14,14), cmap="Greys")

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
conv2d의 shape : (1, 14, 14, 5)
conv2d_img의 shape : (5, 14, 14, 1)


# tensorflow-MNIST with CNN


In [8]:
import tensorflow as tf
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

# 그래프를 초기화해보자! 
tf.reset_default_graph() 

# Data Loading
mnist = input_data.read_data_sets("./data/mnist",one_hot=True)

# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32) # drop out 비율

# Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # -1: 몇장인지 모름    # convolution하려면 무조건 4차원이어야 함!
F1 = tf.Variable(tf.random_normal(shape=[3,3,1,32]))  #  32개의 filter map
    # 여기서는 자비에르초기법 하는거 아님. W의 초기값을 랜덤으로 받는다는 건데 그건 머신러닝때. 지금은 filter를 랜덤으로 잡는 과정이므로 랜덤해야한다.
L1 = tf.nn.conv2d(x_img, F1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)  # convolution 하고 relu 해줘야 함(값 너무 커지지 않게)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
# -> 14*14 크기가 32개 나옴

F2 = tf.Variable(tf.random_normal(shape=[3,3,32,64]))  # 32: 앞에서 나오는 channel 수
    # 3*3 크기의 32 depth를 64번 filter하겠다
L2 = tf.nn.conv2d(L1, F2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)  
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")    
# -> 7*7 크기가 64개 나옴  


# 이렇게 만든 데이터를 FC Layer에 넣어서 학습해야 한다.
L2 = tf.reshape(L2,[-1,7*7*64])

W1 = tf.get_variable("weight1", shape=[7*7*64,256],
                    initializer=tf.contrib.layers.xavier_initializer())
############################################################################

b1 = tf.Variable(tf.random_normal([256]),name="bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W1)+b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_rate) # 256개  output을 다 뽑아내지 않겠다. node를 아예 삭제하는게 아니라 기능을 상실시키는 것
                                         # rate=0 하면 다 살아있는거   0.3하면 30% 죽이는거 -> 다음으로 30프로 죽여서 넘기겠다.
     # drop_rate가 cpu_env에서는 keep_rate
W2 = tf.get_variable("weight2", 
                     shape=[256,256], 
                     initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]),name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
layer2 = tf.nn.dropout(_layer2,keep_prob=keep_rate) 

W3 = tf.get_variable("weight3", 
                     shape=[256,10], 
                     initializer = tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]),name="bias3")  # 맨 마지막에는 dropout해주지 않는다. -> 다 넘겨야 하므로
logit = tf.matmul(layer2,W3)+b3
H = tf.nn.relu(logit)  

# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=Y))  

# train
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)  # 이거도 자주쓰임. 크게 성능의 향상 보기엔 어렵지만 좀더 성능 좋다는게 일반적인 생각

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch =50  # 반복횟수
batch_size = 100

for step in range(num_of_epoch): 
    num_of_iter = int(mnist.train.num_examples/batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)  # x,y 100개씩 떼어오기
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y,
                                                         keep_rate:0.7}) # 30% 끄고 학습해라
    if step%5==0:
        print("cost:{}".format(cost_val))

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
cost:0.2335168868303299
cost:0.1703042984008789
cost:0.10083776712417603
cost:0.20362286269664764
cost:0.15158894658088684
cost:0.2773580849170685
cost:0.1130962073802948
cost:0.372658908367157
cost:0.10209430754184723
cost:0.05403153598308563


In [6]:
# accuracy 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print("정확도:{}".format(sess.run(accuracy, feed_dict={X:mnist.test.images,
                                                       Y:mnist.test.labels,
                                                       keep_rate:1})))

정확도:0.9794999957084656


In [3]:
#### 혜준이거##########
# tensorflow-MNIST with CNN

import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# Graph 초기화
tf.reset_default_graph()

# Placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32)

# Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1]) # X를 이모양으로 바꿔
W1 = tf.Variable(tf.random_normal([3,3,1,32])) # 3,3,1에 filter 32개 
L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], padding="SAME") 
L1 = tf.nn.relu(L1) # convolution을 하면 값이 커지거나 작아지기때문에 relu를 해줍니다.
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME") # pooling도 여기서는 해줄게요
# stride를 2로 잡고 padding을 해주었으니 size가 반으로 줄었겠죠 
# 결과적으로 [None,14,14,32]

W2 = tf.Variable(tf.random_normal([3,3,32,64])) # (새필터가로, 새필터세로, 이전에필터갯수, 새로운필터갯수)
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME") 
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
# stride를 2로 잡고 다시 padding을 해주었으니 size가 또 반으로 줄었겠죠
# 결과적으로 [None,7,7,64]


# 이렇게 만든 데이터를 FC Layer에 넣어서 학습해야 해요!
L2 = tf.reshape(L2,[-1,7*7*64])

W3 = tf.get_variable("weight3", shape=[7*7*64,256],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name= "bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W3) + b1) # 이렇게 하면 256개가 나옴
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_rate) # 위의 256개의 노드의 기능을 상실시키겠다.30%를 죽여서 넘기겠다.
# cpu_env에서는 rate가 아니라 keep_prob으로 keep_rate는 남기고 싶은 퍼센트를 넣어주면 된다


W4 = tf.get_variable("weight4", shape=[256,256],
                     initializer=tf.contrib.layers.xavier_initializer()) # 초기화 방법       
b2 = tf.Variable(tf.random_normal([256]), name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W4) + b2)
layer2 = tf.nn.dropout(_layer2, keep_prob=keep_rate)

W5 = tf.get_variable("weight5", shape=[256,10],
                     initializer=tf.contrib.layers.xavier_initializer()) # 초기화 방법      
b3 = tf.Variable(tf.random_normal([10]), name="bias3")


# Hypothesis
logit = tf.matmul(layer2, W5) + b3
H = tf.nn.relu(logit)


# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
                                                              labels = Y))

# train
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
# Adamoptimizer가 더 좋은 성능을 가진 함수이다!!!

# session, 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch = 30 
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples / batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x,
                                                       Y:batch_y,
                                                       keep_rate:0.7})
        
    if step % 3 == 0:
        print("Cost : {}".format(cost_val))

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
Cost : 0.47341403365135193
Cost : 0.3523519039154053
Cost : 0.29518187046051025
Cost : 0.16579708456993103


KeyboardInterrupt: 